In [1]:
## toolkit libraries to be installed

##data services
from qpds import repo_connector
from qpds import universe
from qpds import calendar
from qpds import history
from qpds import backtest as backtest_data_fetch

#rules and definition
from qidxrules.utils import rule_builder
from qidxdef import index

#backtest calculation
from qpdcalc import backtest as backtest_calc
from qpdcalc.async_task import create_backtest_job

#analytics
from qit import main


## external libraries
import os
import numpy as np
import pandas as pd
import datetime
import warnings
import time


warnings.filterwarnings('ignore')
pd.set_option("expand_frame_repr", False)


In [2]:
#Configure parameters needed for the get universe call
data_envt = 'PROD'
calc_envt = "PROD"
parent_symbol = "SX5P"
ccy = 'EUR'
cal = 'STOXXCAL'
vendor_items = ['RBICS','Sustainalytics','ISS','EconsightMetaverse'] #'RBICS','Sustainalytics','ISS', RBICS_FOCUS
fields = ['TobaccoProdMaxRev','Total ESG Score','Environment Score']

calendar_name = 'STXSQ1'
start_at = datetime.date(2020, 1, 1)
end_at = datetime.date.today()

calc_input_type = 'file'


In [3]:
#authentication service
repo = repo_connector.connect(data_envt)

## ensure you are authenticated to GCP using "gcloud auth application-default login" from command line before using calc service

In [4]:
##indexDefinition : Create & Add rules to index
##################################
# Create Rules #

##selection filters
rule_1 = rule_builder.filter_simple(attribute='environment', operator='>=', value=0,special_inclusions=[str(pd.NA)] ,dataset_id='parent_universe')

rule_2 = rule_builder.filter_simple(attribute='TobaccoProdMaxRev', operator='>=', value=0,special_inclusions=[str(pd.NA),"Not Collected"] ,dataset_id='parent_universe')
rule_3 = rule_builder.filter_simple(attribute='Environment Score', operator='>=', value=50,special_exclusions=['<NA>'],dataset_id = 'parent_universe')
rule_4 = rule_builder.filter_nested_attribute(attribute='rbics_l6', operator = '>=', value= 25, nested_attribute= ['151010151540', '552010351025', '552015202010', '552010351015',
                                                                                                     '552010351030', '552010201510', '151010153025', '101010101510',
                                                                                                     '151010152515', '151010151535', '151010152510', '151010151510',
                                                                                                     '202515101010', '552015201010', '552015201510', '552015202510',
                                                                                                     '552020253010', '551520103010', '552015251510', '551530102010',
                                                                                                     '151010151515', '551020302510', '551020401525', '551520102010',
                                                                                                     '551520151510', '551520251010', '551525301010', '551010151510',
                                                                                                     '551525451010', '552015102525',              
                                                                                                    ],
                                                                            cumulate_nested_attribute= True,dataset_id = 'parent_universe')
rule_5 = rule_builder.weight_by_attribute(attribute='ffmcap',dataset_id = 'parent_universe')
rule_6 = rule_builder.weight_factor(weight_attribute='weight', price_attribute="adjustedOpenPrice", scale_factor_billions=1)
##Index defintion 
idxdef = index.Index()
selection_stage = idxdef.add_stage(name='selection', calendar_name='STXSQ1',price_date="cutoff_date")
selection_stage.add_rules([rule_1,rule_2,rule_3,rule_4])
 
weighting_stage = idxdef.add_stage(name='weighting', calendar_name='STXWQ1',price_date="cutoff_date")
weighting_stage.add_rules([rule_5,rule_6])
 
idxdef.describe()
 


'{"index_parameter": {}, "stages": [{"name": "selection", "calendar_name": "STXSQ1", "price_date": "cutoff_date", "required_data": [], "rules": [{"rule": "filter_simple", "rule_type": 1, "rule_package": "qidxrules", "attribute": "environment", "operator": ">=", "value": 0, "special_inclusions": ["<NA>"], "dataset_id": "parent_universe"}, {"rule": "filter_simple", "rule_type": 1, "rule_package": "qidxrules", "attribute": "TobaccoProdMaxRev", "operator": ">=", "value": 0, "special_inclusions": ["<NA>", "Not Collected"], "dataset_id": "parent_universe"}, {"rule": "filter_simple", "rule_type": 1, "rule_package": "qidxrules", "attribute": "Environment Score", "operator": ">=", "value": 50, "special_exclusions": ["<NA>"], "dataset_id": "parent_universe"}, {"rule": "filter_nested_attribute", "rule_type": 11, "rule_package": "qidxrules", "attribute": "rbics_l6", "operator": ">=", "value": 25, "dataset_id": "parent_universe", "nested_attribute": ["151010151540", "552010351025", "552015202010", 

In [5]:
#get calendar days
some_days = calendar.get_calendar_days(repo,
                                        calendar_name,
                                        start_at,
                                        end_at)
#some_days = some_days[['cuttOffDay','effectiveDay']]
print(some_days)

date_pairs = list(zip(some_days['cuttOffDay'],some_days['effectiveDay']))
print(date_pairs)

   effectiveDay  cuttOffDay implementationDay notReportBeforeDate rebalancingCutOffDate publicationDate rebalancingCutOffDateMDR selectionListDate
0    2020-03-23  2020-02-28        2020-03-20          2020-03-14            2020-03-12      2020-03-13               2020-02-28        2020-03-24
1    2020-06-22  2020-05-29        2020-06-19          2020-06-13            2020-06-11      2020-06-12               2020-05-29        2020-06-23
2    2020-09-21  2020-08-31        2020-09-18          2020-09-12            2020-09-10      2020-09-11               2020-08-31        2020-09-22
3    2020-12-21  2020-11-30        2020-12-18          2020-12-12            2020-12-10      2020-12-11               2020-11-30        2020-12-22
4    2021-03-22  2021-02-26        2021-03-19          2021-03-13            2021-03-11      2021-03-12               2021-02-26        2021-03-23
5    2021-06-21  2021-05-31        2021-06-18          2021-06-12            2021-06-10      2021-06-11               

In [6]:
## running historical index reviews - get universe call & running index deinition in the loop

index_def_result_all =pd.DataFrame() #creating a df to capture the consolidate the output of the review

for (securities_data_cutoff_date,composition_date) in date_pairs: # running for every review date in the backtest period
    
    #getting the data points needed for the selection and weighting using the data services
    result_initial = universe.get(repo, parent_symbol, securities_data_cutoff_date, composition_date,
                    cal, ccy,
                    fields,vendor_items,sid_direct=True)

    result_initial.to_csv(r"C:\Users\sradhakrishnan\OneDrive - ISS\Sriram\New Platform\iStudio\iDK_universe_output.csv",index=False)

    #run the selection and weighting rule defined for the index
    idxdef_result = idxdef.run(result_initial)
    print(composition_date,len(idxdef_result.index),idxdef_result[idxdef_result['exclusion']== False].shape[0])

    #consolidate the backtested reviews
    index_def_result_all = pd.concat([index_def_result_all,idxdef_result])

#print backtested compositions -- only inclusions
print(index_def_result_all[index_def_result_all['exclusion']== False])

#save the backtested selection output including exclusions 
index_def_result_all.to_csv(r"C:\Users\sradhakrishnan\OneDrive - ISS\Sriram\New Platform\iStudio\iDK_selection_output.csv",index=False)


Success!
** Ready to fetch ['iss'] vendor data from SIDS REST API **


KeyboardInterrupt: 

In [8]:
### no need to provide Index Symbol and ISIN while defining variants
### no need to pass the effective date additionally in the backtest data  
input_type = calc_input_type
env = calc_envt
start_date = pd.to_datetime(min(some_days['effectiveDay']),format='%Y-%m-%d')
end_date = datetime.date.today()


backtest_data = index_def_result_all[index_def_result_all['exclusion']== False]
backtest_data['composition_date']=pd.to_datetime(backtest_data['compositionDate'],format='%Y%m%d').dt.date
backtest_data['weight_factor']= backtest_data['weight_factor'].astype(int)

backtest_data = backtest_data.rename(columns={'weight':'weights'})
backtest_data=backtest_data[['stoxxid','composition_date','weight_factor']].rename(columns={'weight_factor':'weightFactor'})



index_compositions = backtest_data.to_dict(orient='list')
print(index_compositions)

##################################
# index level methods #
##################################

# create index parameter
my_index_parameter = index.IndexParameter(index_name="idx sample 230",
                                            base_date= pd.to_datetime(min(some_days['effectiveDay']),format='%Y-%m-%d'),
                                            base_value=1000,
                                            calculation_calendar="STOXXCAL",
                                            weight_adjustment_type="price weighted",
                                            rebalancing_type="None",
                                            parent_index="SXW1E",
                                            review_calendar="STXSQ1")
my_index_parameter.add_variants(index_symbol="DUMMY1",
                                isin="DD1234567890",
                                index_currency="EUR",
                                return_type="pr")
my_index_parameter.add_variants(index_symbol="DUMMY2",
                                isin="XX1234567890",
                                index_currency="USD",
                                return_type="nr")

# create index
my_index = index.Index()

# add index parameter to index
my_index.add_index_parameter(my_index_parameter)

print(my_index.describe())


data = create_backtest_job(env,
                               input_type,
                               my_index,
                               index_compositions,
                               start_date,
                               end_date)
    # Print the batch_id and task name
print({'batch_id': data['batch_id'],
        'message': 'Job started.'
        })

batchid = data['batch_id']


{'stoxxid': ['026256', '587183', '619545', '636679', '663955', '664706', '682150', '772868', 'AAPL', 'AMD', 'AU10GL', 'B0JDQD', 'BBY', 'GB60DA', 'INTC', 'LVS', 'MGG', 'NVDA', 'PH256', 'US20PD', 'US61PI', '026256', '587183', '619545', '636679', '663955', '664706', '682150', '772868', 'AAPL', 'AMD', 'AU10GL', 'B0JDQD', 'BBY', 'GB60DA', 'INTC', 'LVS', 'MGG', 'NVDA', 'PH231', 'PH256', 'US20PD', 'US61PI', 'US813Y', '026256', '587183', '619545', '636679', '663955', '664268', '664706', '686930', '772868', 'AAPL', 'AMD', 'AU10GL', 'B0JDQD', 'BBY', 'GB60DA', 'INTC', 'LVS', 'MGG', 'NVDA', 'PH231', 'PH256', 'US20PD', 'US61PI', 'US813Y', '026256', '587183', '619545', '636679', '663955', '664268', '664706', '686930', '772868', 'AAPL', 'AMD', 'AU10GL', 'B0JDQD', 'BBY', 'GB60DA', 'INTC', 'LVS', 'MGG', 'NVDA', 'PH256', 'US20PD', 'US61PI', 'US813Y', '026256', '587183', '619545', '636679', '663955', '664268', '664706', '772868', 'AAPL', 'AMD', 'AU10GL', 'B0JDQD', 'BBY', 'GB60DA', 'HK209C', 'INTC', 'LVS'